<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ee660c23a7a94cfb103b90b0f39bae7df4184703d9125691f06d061796dbbf61
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 11:17:31
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -43.86 K (-0.31%)
Current PnL: -19.95 L (-13.29%)
CY Booked + Current PnL: -7.85 L (-5.23%)
-------------------
Total profit:  2.17 L
Total loss:  -22.12 L
-------------------
Total Booked + Current PnL: 19.13 L (15.54%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.54%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.13 L (61.49%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-9.53,66.0,X-LC,3.17,226028.0,15662.0,13110.0,0.03,7.45,5.80,13.68,37.0,1.19,1.60,25.20,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.98,60.0,X-MC,2.79,215940.0,24198.0,13561.0,0.59,12.62,6.28,19.69,99.0,1.78,1.53,20.28,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-9.04,44.0,X-MC,3.32,89790.0,-888.0,15794.0,-0.61,-0.98,17.59,16.44,101.0,-0.06,0.64,19.00,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,95.52,48.0,M-SC,1.98,84226.0,-16551.0,16643.0,0.17,-16.42,19.76,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,61.0,X-LC,1.98,202715.0,2577.0,18305.0,-0.34,1.29,9.03,10.44,4.0,0.14,1.44,6.92,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-195.45,79.0,M-SC,24.41,164466.0,15246.0,128941.0,4.78,10.22,78.40,96.62,208.0,0.12,1.17,77.99,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.07,55.0,M-LC,11.05,213454.0,8378.0,104187.0,1.77,4.09,48.81,54.89,52.0,0.08,1.51,29.17,X5K,ATH,METALS
36,IEX,219.00,-30.39,62.0,H-SC,17.43,199058.0,-896.0,98116.0,1.50,-0.45,49.29,48.63,115.0,-0.01,1.41,11.05,XR,NTT,MISC
73,TATAELXSI,9161.00,-22.37,54.0,H-MC,7.50,104975.0,-26860.0,69084.0,0.99,-20.37,65.81,32.03,98.0,-0.39,0.74,17.58,OX40N,NTT,IT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDUNILVR,2922.00,-10.50,44.0,X-LC,7.66,251420.0,-5643.0,40780.0,-3.36,-2.20,16.22,13.67,24.0,-0.14,1.78,17.77,XY25,NTT,FMCG
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.19,221280.0,-42085.0,155427.0,-3.24,-15.98,70.24,43.04,84.0,-0.27,1.57,2.73,XY25,ATH,FMCG
22,DABUR,735.00,-10.14,47.0,X-MC,3.90,243276.0,-4792.0,113196.0,-1.92,-1.93,46.53,43.70,102.0,-0.04,1.72,12.23,XY24,BTT,FMCG
7,ASIANTILES,137.00,7116.67,64.0,L-SC,7.20,79156.0,-14654.0,91275.0,-1.84,-15.62,115.31,81.67,269.0,-0.16,0.56,53.29,XR,NTT,CERAMICS
26,GREENPANEL,537.00,230.10,53.0,M-SC,5.37,147400.0,-35678.0,114662.0,-1.80,-19.49,77.79,43.14,230.0,-0.31,1.04,38.65,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.94,47.0,M-MC,10.01,223722.0,-1240.0,168888.0,0.45,-0.55,75.49,74.53,192.0,-0.01,1.58,31.52,XY24,BTT,STEEL
36,IEX,219.0,-30.39,62.0,H-SC,17.43,199058.0,-896.0,98116.0,1.50,-0.45,49.29,48.63,115.0,-0.01,1.41,11.05,XR,NTT,MISC
87,WIPRO,420.0,-13.79,48.0,M-LC,5.92,151503.0,558.0,108900.0,0.02,0.37,71.88,72.51,53.0,0.01,1.07,6.41,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,68.0,H-SC,15.26,140427.0,-2052.0,21570.0,0.21,-1.44,15.36,13.70,163.0,-0.10,0.99,65.74,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,58.0,H-SC,2.37,228006.0,-41661.0,78001.0,-0.82,-15.45,34.21,13.47,138.0,-0.53,1.62,15.93,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,95.52,48.0,M-SC,1.98,84226.0,-16551.0,16643.0,0.17,-16.42,19.76,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.37,54.0,H-MC,7.50,104975.0,-26860.0,69084.0,0.99,-20.37,65.81,32.03,98.0,-0.39,0.74,17.58,OX40N,NTT,IT
67,SIS,528.0,2072.54,52.0,H-SC,3.24,87682.0,-23350.0,46962.0,-0.32,-21.03,53.56,21.26,156.0,-0.50,0.62,18.47,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,48.0,M-LC,5.68,205865.0,2695.0,39444.0,-0.16,1.33,19.16,20.74,55.0,0.07,1.46,6.94,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.07,55.0,M-LC,11.05,213454.0,8378.0,104187.0,1.77,4.09,48.81,54.89,52.0,0.08,1.51,29.17,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.91,59.97,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,60.0,M-LC,9.13,162182.0,21342.0,23143.0,-0.15,15.15,14.27,31.59,66.0,0.92,1.15,45.66,XY24,NTT,BANKS
87,WIPRO,420.00,-13.79,48.0,M-LC,5.92,151503.0,558.0,108900.0,0.02,0.37,71.88,72.51,53.0,0.01,1.07,6.41,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-19.25,70.0,H-MC,14.28,206969.0,27161.0,85251.0,-0.71,15.11,41.19,62.52,88.0,0.32,1.47,54.34,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103928.0,12430.0,66327.0,-0.98,13.58,63.82,86.07,223.0,0.19,0.74,50.92,XR,NTT,DURABLES
32,HINDZINC,730.22,31.07,55.0,M-LC,11.05,213454.0,8378.0,104187.0,1.77,4.09,48.81,54.89,52.0,0.08,1.51,29.17,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.91,59.97,MH,ATH,POWER
87,WIPRO,420.00,-13.79,48.0,M-LC,5.92,151503.0,558.0,108900.0,0.02,0.37,71.88,72.51,53.0,0.01,1.07,6.41,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
24,DMART,5391.45,-17.07,34.0,X-LC,10.16,101453.0,-1133.0,27940.0,-0.96,-1.10,27.54,26.13,38.0,-0.04,0.72,24.19,X40N,ATH,FMCG
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51345.0,-13661.0,164592.0,-0.23,-21.01,320.56,232.18,198.0,-0.08,0.36,0.15,XY24,NTT,MISC
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.19,221280.0,-42085.0,155427.0,-3.24,-15.98,70.24,43.04,84.0,-0.27,1.57,2.73,XY25,ATH,FMCG
52,PGHH,17907.65,-29.49,42.0,X-MC,3.23,204150.0,3330.0,64471.0,-0.69,1.66,31.58,33.76,80.0,0.05,1.45,6.94,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.54,51.0,H-LC,0.80,158250.0,-27845.0,75327.0,0.48,-14.96,47.60,25.51,15.0,-0.37,1.12,16.62,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,61.0,X-LC,1.98,202715.0,2577.0,18305.0,-0.34,1.29,9.03,10.44,4.0,0.14,1.44,6.92,X40,NTT,FMCG
85,VOLTAS,1530.00,2.98,60.0,X-MC,2.79,215940.0,24198.0,13561.0,0.59,12.62,6.28,19.69,99.0,1.78,1.53,20.28,XY25,NTT,AC
29,HAVELLS,2069.16,-5.47,49.0,X-MC,2.88,234328.0,-13403.0,88459.0,0.38,-5.41,37.75,30.30,92.0,-0.15,1.66,7.16,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.53,66.0,X-LC,3.17,226028.0,15662.0,13110.0,0.03,7.45,5.80,13.68,37.0,1.19,1.60,25.20,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51345.0,-13661.0,164592.0,-0.23,-21.01,320.56,232.18,198.0,-0.08,0.36,0.15,XY24,NTT,MISC
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.19,221280.0,-42085.0,155427.0,-3.24,-15.98,70.24,43.04,84.0,-0.27,1.57,2.73,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,48.0,X-MC,2.91,184360.0,-53591.0,206244.0,-0.65,-22.52,111.87,64.15,174.0,-0.26,1.31,3.42,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-14.27,47.0,X-MC,5.75,180810.0,-19696.0,116858.0,0.05,-9.82,64.63,48.45,90.0,-0.17,1.28,4.27,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51345.0,-13661.0,164592.0,-0.23,-21.01,320.56,232.18,198.0,-0.08,0.36,0.15,XY24,NTT,MISC
51,PAGEIND,51769.93,-26.93,42.0,X-MC,6.76,82550.0,-960.0,20992.0,-0.45,-1.15,25.43,23.98,82.0,-0.05,0.58,4.60,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.30,45.0,X-SC,4.68,82767.0,-62393.0,138320.0,-0.79,-42.98,167.12,52.31,136.0,-0.45,0.59,10.79,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.04,44.0,X-MC,3.32,89790.0,-888.0,15794.0,-0.61,-0.98,17.59,16.44,101.0,-0.06,0.64,19.00,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.43,96869.0,-31801.0,77098.0,-0.04,-24.72,79.59,35.20,219.0,-0.41,0.69,11.18,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.40,59.0,X-LC,12.33,289633.0,-56323.0,123036.0,0.26,-16.28,42.48,19.28,1.0,-0.46,2.05,7.07,X40,ATH,IT
40,INFY,2275.00,-16.73,59.0,X-LC,7.17,324953.0,12065.0,159617.0,-0.19,3.86,49.12,54.87,3.0,0.08,2.30,10.76,X40,BTT,IT
42,ITC,452.00,-37.08,61.0,X-LC,1.98,202715.0,2577.0,18305.0,-0.34,1.29,9.03,10.44,4.0,0.14,1.44,6.92,X40,NTT,FMCG
84,VBL,671.64,-18.39,53.0,X-LC,9.88,294150.0,-21692.0,134368.0,-0.97,-6.87,45.68,35.67,5.0,-0.16,2.08,6.03,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,48.0,H-LC,6.31,249240.0,-12379.0,131599.0,0.21,-4.73,52.80,45.57,7.0,-0.09,1.77,6.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7116.67,64.0,L-SC,7.20,79156.0,-14654.0,91275.0,-1.84,-15.62,115.31,81.67,269.0,-0.16,0.56,53.29,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,46.0,L-SC,26.93,79776.0,-33773.0,73825.0,-1.26,-29.74,92.54,35.27,268.0,-0.46,0.57,95.25,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,48.0,H-SC,9.55,91410.0,-6570.0,28173.0,-0.78,-6.71,30.82,22.05,152.0,-0.23,0.65,32.87,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103928.0,12430.0,66327.0,-0.98,13.58,63.82,86.07,223.0,0.19,0.74,50.92,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.91,59.97,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-195.45,79.0,M-SC,24.41,164466.0,15246.0,128941.0,4.78,10.22,78.40,96.62,208.0,0.12,1.17,77.99,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,68.0,H-SC,15.26,140427.0,-2052.0,21570.0,0.21,-1.44,15.36,13.70,163.0,-0.10,0.99,65.74,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.73,70.0,M-SC,9.60,83702.0,-17457.0,57461.0,0.05,-17.26,68.65,39.54,202.0,-0.30,0.59,18.03,AR,ATH,AUTO
5,ANGELONE,3033.00,15.82,67.0,X-SC,6.41,204541.0,13535.0,44160.0,-0.66,7.09,21.59,30.21,157.0,0.31,1.45,29.45,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-56.04,66.0,H-SC,2.90,252515.0,-33990.0,291327.0,-0.90,-11.86,115.37,89.82,134.0,-0.12,1.79,33.60,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.42
1,25,43.80
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.43
LC    32.77
MC    22.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.30
X40      16.05
XY25     12.24
XR       11.29
X40N      9.31
AR        7.91
OX40N     7.63
X200      1.78
X5K       1.51
SR        1.09
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.59
X-LC    21.88
X-MC    17.05
M-SC    12.74
M-LC     5.19
H-LC     4.67
X-SC     4.63
H-MC     3.89
M-MC     1.58
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.62,-4.46,38.56
IT,12.75,-16.66,79.56
FINANCE,12.20,-9.95,59.79
MISC,6.93,-17.22,82.18
BANKS,6.61,-6.95,63.21
PAINTS,5.63,-16.22,33.54
ELECTRICAL,5.32,-9.67,48.15
INSURANCE,3.89,-0.70,38.25
AUTO,2.81,-44.75,105.16


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3297353.0,22
XR,1317920.0,14
AR,1178096.0,9
X40,779560.0,12
X40N,614513.0,8
OX40N,570179.0,10
XY25,455658.0,8
SR,267341.0,2
X5K,104187.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3389856.0,25
M-SC,1210248.0,15
X-MC,1129892.0,13
X-LC,1048306.0,13
X-SC,527231.0,6
H-MC,331345.0,3
H-LC,279061.0,3
M-LC,275674.0,4
L-SC,254460.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1170456.0      6
           AR         846986.0      5
           XR         784325.0      7
M-SC       XY24       695059.0      6
X-MC       XY24       591359.0      4
X-LC       X40        449775.0      5
           XY24       306846.0      2
X-SC       X40N       285541.0      4
H-SC       SR         267341.0      2
           OX40N      264289.0      4
X-MC       X40        252687.0      6
X-LC       X40N       212114.0      3
H-LC       AR         206926.0      2
H-MC       XY24       177010.0      1
X-MC       XY25       168988.0      2
M-MC       XY24       168888.0      1
L-SC       XR         165100.0      2
X-SC       XY24       164592.0      1
M-SC       OX40N      147446.0      4
           XY25       128941.0      1
           AR         124184.0      2
X-MC       X40N       116858.0      1
M-SC       XR         114618.0      2
M-LC       XR         108900.0      1
           X5K        104187.0      1
L-SC       OX40N       89360.0      1
H-MC       XR          85251.0      1
X-LC       XY25        79571.0      3
X-SC       X40         77098.0      1
H-LC       X200        72135.0      1
H-MC       OX40N       69084.0      1
L-MC       XR          59726.0      1
H-SC       MH          56459.0      1
M-LC       XY25        39444.0      1
L-LC       XY25        38714.0      1
M-LC       XY24        23143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
